__Optimization Modeling using `PuLP`__

https://medium.com/analytics-vidhya/optimization-modelling-in-python-scipy-pulp-and-pyomo-d392376109f4

In [1]:
%load_ext nb_black

import sys
import numpy as np
import pandas as pd
import pulp

customer_demand = {1: 80, 2: 270, 3: 250, 4: 160, 5: 180}

factory_capacity = {1: 500, 2: 500, 3: 500}

customers = customer_demand.keys()

factories = factory_capacity.keys()

transportation_cost = {
    (1, 1): 4,
    (1, 2): 6,
    (1, 3): 9,
    (2, 1): 5,
    (2, 2): 4,
    (2, 3): 7,
    (3, 1): 6,
    (3, 2): 3,
    (3, 3): 3,
    (4, 1): 8,
    (4, 2): 5,
    (4, 3): 3,
    (5, 1): 10,
    (5, 2): 8,
    (5, 3): 4,
}

<IPython.core.display.Javascript object>

In [2]:
goods = pulp.LpVariable.dicts(
    "amount of goods",
    ((customer, factory) for customer in customers for factory in factories),
    lowBound=0,
    cat="Continuous",
)

<IPython.core.display.Javascript object>

In [34]:
objective = pulp.LpAffineExpression(
    e=[
        (goods[customer, factory], transportation_cost[customer, factory])
        for customer, factory in goods
    ],
    name="Objective function",
)

model = pulp.LpProblem(name="Transportation cost minimization", sense=pulp.LpMinimize)
model += pulp.lpSum(objective)

<IPython.core.display.Javascript object>

In [35]:
# Constraint: sum of goods == customer demand
for customer in customers:
    expression = pulp.LpAffineExpression(
        e=[
            (goods[customer, factory], 1)
            for factory in factories
            if (customer, factory) in goods
        ]
    )
    constraint = pulp.LpConstraint(
        e=pulp.lpSum(tmpExpression),
        sense=pulp.LpConstraintEQ,
        rhs=customer_demand[customer],
    )
    model.addConstraint(constraint)

    # Constraint: sum of goods <= factory capacity
    for factory in factories:
        tmpExpression = pulp.LpAffineExpression(
            e=[
                (goods[customer, factory], 1)
                for factory in factories
                if (customer, factory) in goods
            ]
        )
        tmpConstraint = pulp.LpConstraint(
            e=pulp.lpSum(tmpExpression),
            sense=pulp.LpConstraintLE,
            rhs=factory_capacity[factory],
        )
        model.addConstraint(tmpConstraint)

<IPython.core.display.Javascript object>

In [36]:
results = model.solve()

<IPython.core.display.Javascript object>

In [37]:
results

1

<IPython.core.display.Javascript object>

In [38]:
if model.status == 1:
    print(f"Solution is optimal: {pulp.LpStatus[model.status]}")
else:
    print(f"Failed to find solution: {pulp.LpStatus[model.status]}")
print(f"Objective function value = {pulp.value(model.objective)}")

EPS = 1.0e-6

for (customer, factory) in goods:
    customer_amount = goods[customer, factory].varValue
    if customer_amount > EPS:
        print(
            f"Sending quantity {customer_amount:>5} from factory {factory} to customer {customer}"
        )

Solution is optimal: Optimal
Objective function value = 3420.0
Sending quantity 270.0 from factory 1 to customer 1
Sending quantity 250.0 from factory 2 to customer 2
Sending quantity 160.0 from factory 2 to customer 3
Sending quantity 180.0 from factory 3 to customer 4
Sending quantity  80.0 from factory 3 to customer 5


<IPython.core.display.Javascript object>